In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

sys.path.append("../core")
sys.path.append("../core/simulations")

import simulation_library, potentials, integrators, data_logging, visuals

from data import gen_double_moon_samples
from losses import getLoss
from trainer import Trainer
from optimizers import getOpt
from network_base import RealNVP
from network_logging import *

plt.style.use("ggplot")

## Crescent Moon Example

In [ ]:
def crescent_moon_example():
    data = gen_double_moon_samples(10000)
    loss = getLoss().basic_loss()
    opt = getOpt().rmsprop(1e-4)
    model = RealNVP(loss, opt, model_name='double_moon')
    model = LogLoss(model)
    model = LogTargetPlot(model)
    model = Checkpointing(model)
    trainer = Trainer(model, data)
    trainer.train(15)
    return trainer

In [ ]:
trained = crescent_moon_example()

## Double Well Potential Example

In [ ]:
def double_well_potential(freq = 100):
    system_builder = simulation_library.SystemFactory()
    coords = []
    for i, s in enumerate([[-2, 0], [-1, 0], [1, 0], [2, 0]]):
        system = simulation_library.System(dim = 2)
        wca = potentials.WCAPotential(1, 1)
        system.central_potential = potentials.DoubleWellPotential(a = 1,
                                                                  b = 6, 
                                                                  c = 1,
                                                                  d = 1)
        
        system.add_particle(simulation_library.Particle(wca, np.array(s)))
        system.get_integrator("metropolis", None, temp = .5)
        coords_logger = data_logging.CoordinateLogger(system, freq)
        energy_logger = data_logging.EnergyLogger(system, freq)
        system.registerObserver(coords_logger)
        system.registerObserver(energy_logger)
        system.run(5000 * freq)
        c = np.array(coords_logger.coordinates, dtype=np.float32).squeeze()[:-freq]
        if(len(coords) == 0):
            coords = c
        else:
            coords = np.concatenate((coords, c), axis=0)
    
    loss = getLoss().basic_loss()
    opt = getOpt().rmsprop(1e-4)
    model = RealNVP(loss, opt, model_name="double_well")
    model = LogLoss(model)
    model = LogTargetPlot(model)
    model = Checkpointing(model)
    trainer = Trainer(model, coords)
    trainer.train(10)
    return trainer, system

In [ ]:
trained_dw, dw_sys = double_well_potential()

In [ ]:
out = np.expand_dims(trained_dw.model.forward_sample(10000).numpy(), axis=1)

In [ ]:
ani = visuals.make_2D_traj_potential(out,
                              potential=dw_sys.central_potential,
                              xlim = [-6, 6],
                              ylim = [-8, 8],
                              cutoff = 10)

ani

In [ ]:
targets = trained_dw.model.forward_sample(1000).numpy()
gauss = trained_dw.model.backward_sample(targets).numpy().T

In [3]:
loss = getLoss().maximum_likelihood_loss()
opt = getOpt().rmsprop(1e-4)
model = RealNVP(loss, opt, model_name="asdfasdf")

In [9]:
print(model.flow.bijector.inverse_log_det_jacobian.__doc__)

Returns the (log o det o Jacobian o inverse)(y).

    Mathematically, returns: `log(det(dX/dY))(Y)`. (Recall that: `X=g^{-1}(Y)`.)

    Note that `forward_log_det_jacobian` is the negative of this function,
    evaluated at `g^{-1}(y)`.

    Args:
      y: `Tensor`. The input to the 'inverse' Jacobian determinant evaluation.
      event_ndims: Number of dimensions in the probabilistic events being
        transformed. Must be greater than or equal to
        `self.inverse_min_event_ndims`. The result is summed over the final
        dimensions to produce a scalar Jacobian determinant for each event, i.e.
        it has shape `rank(y) - event_ndims` dimensions.
      name: The name to give this op.
      **kwargs: Named arguments forwarded to subclass implementation.

    Returns:
      ildj: `Tensor`, if this bijector is injective.
        If not injective, returns the tuple of local log det
        Jacobians, `log(det(Dg_i^{-1}(y)))`, where `g_i` is the restriction
        of `g` to t

In [17]:
target = model.forward_sample(1000)
gauss = model.backward_sample(target)
tf.reduce_mean(model.flow.bijector.inverse_log_det_jacobian(target, 2))

<tf.Tensor: shape=(), dtype=float32, numpy=19.3912>

In [19]:
model.flow.distribution.sample(1000)

<tf.Tensor: shape=(1000, 2), dtype=float32, numpy=
array([[ 0.42140055, -0.43197063],
       [-0.88724947, -0.37996253],
       [ 1.3165752 , -0.7802854 ],
       ...,
       [-0.3452824 ,  1.6943659 ],
       [ 0.95830786,  0.6928664 ],
       [ 1.4096792 , -0.56727624]], dtype=float32)>

## Mueller Potential Example

In [ ]:
def mueller_potential(freq = 500):
    system_mp = simulation_library.System(dim = 2)
    coords = []
    for i, s in enumerate([[1, 0], [0, 0.5], [-0.5, 1.5]]):
        system_mp.central_potential = potentials.MuellerPotential(alpha = 0.1, 
                                                                  A = [-200, -100, -170, 15],
                                                                  a = [-1, -1, -6.5, 0.7],
                                                                  b = [0, 0, 11, 0.6],
                                                                  c = [-10, -10, -6.5, 0.7],
                                                                  xj = [1, 0, -0.5, -1],
                                                                  yj = [0, 0.5, 1.5, 1]
                                                                 )
        wca = potentials.WCAPotential(1,1)
        system_mp.add_particle(simulation_library.Particle(wca, np.array([0, 1.5])))

        system_mp.get_integrator("verlet", dt = 0.0001)
        system_mp.get_thermostat("anderson", T = 1, freq = 100, colisions = 0.01)
        coords_logger_mp = data_logging.CoordinateLogger(system_mp, freq)
        energy_logger_mp = data_logging.EnergyLogger(system_mp, freq)
        system_mp.registerObserver(coords_logger_mp)
        system_mp.registerObserver(energy_logger_mp)
        system_mp.run(5000 * freq)
        c = np.array(coords_logger_mp.coordinates, dtype=np.float32).squeeze()[: - freq]
        if(len(coords) == 0):
            coords = c
        else:
            for k in range(c.shape[1]):
                coords = np.concatenate((coords, c[:, k, :]), axis=0)        
    
    loss = getLoss().basic_loss()
    opt = getOpt().rmsprop(1e-4)
    model = RealNVP(loss, opt, model_name='mueller_potential')
    model = LogLoss(model)
    model = LogTargetPlot(model)
    model = Checkpointing(model)
    trainer = Trainer(model, coords)
    trainer.train(20)
    return trainer

In [ ]:
trained_mp = mueller_potential()